In [2]:
import sqlite3
import pandas as pd
import numpy as np
import hashlib
from pandasql import sqldf
from geopy.distance import great_circle

Part 1. Establishing Database

In [3]:
conn = sqlite3.connect("cs.db")

In [4]:
cur = conn.cursor()

In [29]:
#Tables on db
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'",conn)

,name
0,fields
1,geo
2,inst
3,user


In [50]:
inst = pd.read_sql("SELECT * FROM inst",conn)

In [31]:
#conn.close()

Part 2. User Inputs

Preferences:
Input provided: Importance from scale 1 to 10
Output created: Allocation Percentage

In [32]:
#a, b, c represents importance user provides for each preference
a=4
b=5
c=6
raw_weights = np.array([a, b, c])
sum_weights = np.sum(raw_weights)
perc_weights = raw_weights/sum_weights

Part 2a. User ID Creation
Input provided: email
Output created: 1) user_id 2) inserts user_id and email into "user" table
Method: function (user_id_generator)

In [33]:
#Scenario 1 Testing
email = 'mkjones@gatech.edu'
user_id = int(hashlib.sha256(email.encode('utf-8')).hexdigest(), 16) % 10**8 # generate a unique numeric ID

In [34]:
user_id

9987952

In [35]:
#Scenario #2
#trying different email
email2 = 'michaelk.Jones@outlook.com'
user_id3 = int(hashlib.sha256(email2.encode('utf-8')).hexdigest(), 16) % 10**8 # generate a unique numeric ID
user_id3

80449489

In [36]:
#Scenario 3
#trying same email as scenario 2 but with all lowercase
email3 = 'michaelk.jones@outlook.com'
user_id4 = int(hashlib.sha256(email3.encode('utf-8')).hexdigest(), 16) % 10**8 # generate a unique numeric ID
user_id4

87012218

In [37]:
#Scenario 4
#same email as scenario 2 but added lower to have negate capitalization effects
email2 = 'michaelk.Jones@outlook.com'
user_id5 = int(hashlib.sha256(email2.lower().encode('utf-8')).hexdigest(), 16) % 10**8 # generate a unique numeric ID
user_id5

87012218

In [38]:
def user_id_generator(email):
    id = int(hashlib.sha256(email.lower().encode('utf-8')).hexdigest(), 16) % 10**8 # generate a unique numeric ID
    return id

In [39]:
#test case of function
user_id_generator(email = 'test123@outlook.com')

83932234

In [40]:
email = 'test123@outlook.com'
user_id = user_id_generator(email = 'test123@outlook.com')

if pd.read_sql(f"SELECT * FROM user WHERE user_id ={user_id}",conn).empty:
    cur.execute(f"INSERT INTO user values('{email}',{user_id})")
else:
    print("Email exists")

In [42]:
pd.read_sql(f"SELECT * FROM user",conn)

,email,user_id
0,mkjones@gatech.edu,9987952
1,mykek.jones@gmail.com,95829733
2,michaelk.jones@outlook.com,87012218
3,test1@GMAIL.COM,65768313
4,test1@gmail.com,45591513


Part 2.b: Zipcode
user provided: zipcode
output created: dataframe of distance in miles per institution

In [9]:
geo = pd.read_sql("SELECT * FROM geo",conn)

In [16]:
geo.printSchema()

AttributeError: 'DataFrame' object has no attribute 'printSchema'

In [45]:
input_zip = "30315"

In [46]:
lat =float(list(geo[geo['zip']==input_zip]['lat'])[0])
lon =float(list(geo[geo['zip']==input_zip]['lng'])[0])
print(f"lat, lon: ({lat}, {lon})")

lat, lon: (33.70312, -84.38272)


In [51]:
inst[["UNITID","LATITUDE","LONGITUDE"]]

,UNITID,LATITUDE,LONGITUDE
0,100654,34.783368,-86.568502
1,100663,33.505697,-86.799345
2,100690,32.362609,-86.17401
3,100706,34.724557,-86.640449
4,100724,32.364317,-86.295677
...,...,...,...
6676,49576719,41.31559,-76.02235
6677,49576720,39.92142,-76.71277
6678,49576721,40.0466,-75.5302
6679,49576722,40.19437,-76.72659


In [57]:
## Function to use to find the distance of each university in miles
def inst_dist (lat,lon):
    idx = inst.apply(lambda x: great_circle((x["LATITUDE"], x["LONGITUDE"]), (lat, lon)).miles, axis=1)
    column_name = f"{user_id}_DISTANCE_MI"  # Adding user_id to the column title
    return inst.loc[:, ['UNITID']].assign(**{column_name: idx})

In [59]:
def inst_dist (lat,lon):
    idx = inst.apply(lambda x: great_circle((x["LATITUDE"], x["LONGITUDE"]), (lat, lon)).miles, axis=1)
    column_name = f"{user_id}_DISTANCE_MI"  # Adding user_id to the column title
    return inst.loc[:, ['UNITID']].assign(dist_miles = idx)

In [61]:
#Distances of each university to the input zip
dist_temp_table = inst_dist(lat,lon)
type(dist_temp_table)
dist_temp_table
inst = pd.merge(inst, dist_temp_table, on='UNITID')

In [5]:
pd.read_sql("SELECT * FROM inst",conn)

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,INSTURL,NPCURL,HCM2,...,OMENRUP_PELL_NFT_POOLED_SUPP,OMENRYP_PELL_FT_POOLED_SUPP,OMENRAP_PELL_FT_POOLED_SUPP,OMAWDP8_PELL_FT_POOLED_SUPP,OMENRUP_PELL_FT_POOLED_SUPP,OMENRYP_PELL_PT_POOLED_SUPP,OMENRAP_PELL_PT_POOLED_SUPP,OMAWDP8_PELL_PT_POOLED_SUPP,OMENRUP_PELL_PT_POOLED_SUPP,GT_THRESHOLD_P6_SUPP
0,100654,00100200,001002,Alabama A & M University,Normal,AL,35762,www.aamu.edu/,www.aamu.edu/admissions-aid/tuition-fees/net-p...,0,...,0.2803,0.0116,0.3329,0.2958,0.3597,0.0056,0.3889,0.1833,0.4222,0.4606
1,100663,00105200,001052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,https://www.uab.edu/,https://tcc.ruffalonl.com/University of Alabam...,0,...,0.211,0.0149,0.2773,0.5243,0.1835,0.0211,0.277,0.4354,0.2665,0.659
2,100690,02503400,025034,Amridge University,Montgomery,AL,36117-3553,www.amridgeuniversity.edu/,www2.amridgeuniversity.edu:9091/,0,...,0.468,0,0.2798,0.2523,0.4679,0,0.186,0.4186,0.3953,0.5132
3,100706,00105500,001055,University of Alabama in Huntsville,Huntsville,AL,35899,www.uah.edu/,finaid.uah.edu/,0,...,0.187,0.0098,0.3154,0.4961,0.1787,0.0105,0.3632,0.3474,0.2789,0.717
4,100724,00100500,001005,Alabama State University,Montgomery,AL,36104-0271,www.alasu.edu/,www.alasu.edu/cost-aid/tuition-costs/net-price...,0,...,0.2026,0.0097,0.3904,0.3061,0.2938,0.0778,0.4444,0.1111,0.3667,0.382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6676,49576719,00332917,003329,Pennsylvania State University-Penn State Wilke...,Lehman,PA,18627-0217,wilkesbarre.psu.edu/,None,0,...,None,None,None,None,None,None,None,None,None,None
6677,49576720,00332918,003329,Pennsylvania State University-Penn State York,York,PA,17403-3326,york.psu.edu/,None,0,...,None,None,None,None,None,None,None,None,None,None
6678,49576721,00332919,003329,Pennsylvania State University-Penn State Great...,Malvern,PA,19355-1488,greatvalley.psu.edu/,None,0,...,None,None,None,None,None,None,None,None,None,None
6679,49576722,00332920,003329,Pennsylvania State University-Penn State Harri...,Middletown,PA,17057-4846,harrisburg.psu.edu/,None,0,...,None,None,None,None,None,None,None,None,None,None


In [6]:
def transform_zip_to_dist_col(zipcode):
    geo = pd.read_sql("SELECT * FROM geo",conn)
    inst = pd.read_sql("SELECT * FROM inst",conn)
    lat =float(list(geo[geo['zip']==zipcode]['lat'])[0])
    lon =float(list(geo[geo['zip']==zipcode]['lng'])[0])
    idx = inst.apply(lambda x: great_circle((x["LATITUDE"], x["LONGITUDE"]), (lat, lon)).miles, axis=1)
    distDF = inst.loc[:, ['UNITID']].assign(dist_miles = idx)
    inst = pd.merge(inst, distDF, on='UNITID')
    return inst

In [7]:
test = transform_zip_to_dist_col('30097')
test

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,INSTURL,NPCURL,HCM2,...,OMENRYP_PELL_FT_POOLED_SUPP,OMENRAP_PELL_FT_POOLED_SUPP,OMAWDP8_PELL_FT_POOLED_SUPP,OMENRUP_PELL_FT_POOLED_SUPP,OMENRYP_PELL_PT_POOLED_SUPP,OMENRAP_PELL_PT_POOLED_SUPP,OMAWDP8_PELL_PT_POOLED_SUPP,OMENRUP_PELL_PT_POOLED_SUPP,GT_THRESHOLD_P6_SUPP,dist_miles
0,100654,00100200,001002,Alabama A & M University,Normal,AL,35762,www.aamu.edu/,www.aamu.edu/admissions-aid/tuition-fees/net-p...,0,...,0.0116,0.3329,0.2958,0.3597,0.0056,0.3889,0.1833,0.4222,0.4606,147.593425
1,100663,00105200,001052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,https://www.uab.edu/,https://tcc.ruffalonl.com/University of Alabam...,0,...,0.0149,0.2773,0.5243,0.1835,0.0211,0.277,0.4354,0.2665,0.659,156.540810
2,100690,02503400,025034,Amridge University,Montgomery,AL,36117-3553,www.amridgeuniversity.edu/,www2.amridgeuniversity.edu:9091/,0,...,0,0.2798,0.2523,0.4679,0,0.186,0.4186,0.3953,0.5132,164.191256
3,100706,00105500,001055,University of Alabama in Huntsville,Huntsville,AL,35899,www.uah.edu/,finaid.uah.edu/,0,...,0.0098,0.3154,0.4961,0.1787,0.0105,0.3632,0.3474,0.2789,0.717,150.129854
4,100724,00100500,001005,Alabama State University,Montgomery,AL,36104-0271,www.alasu.edu/,www.alasu.edu/cost-aid/tuition-costs/net-price...,0,...,0.0097,0.3904,0.3061,0.2938,0.0778,0.4444,0.1111,0.3667,0.382,169.201972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6676,49576719,00332917,003329,Pennsylvania State University-Penn State Wilke...,Lehman,PA,18627-0217,wilkesbarre.psu.edu/,None,0,...,None,None,None,None,None,None,None,None,None,670.943595
6677,49576720,00332918,003329,Pennsylvania State University-Penn State York,York,PA,17403-3326,york.psu.edu/,None,0,...,None,None,None,None,None,None,None,None,None,577.729643
6678,49576721,00332919,003329,Pennsylvania State University-Penn State Great...,Malvern,PA,19355-1488,greatvalley.psu.edu/,None,0,...,None,None,None,None,None,None,None,None,None,630.970517
6679,49576722,00332920,003329,Pennsylvania State University-Penn State Harri...,Middletown,PA,17057-4846,harrisburg.psu.edu/,None,0,...,None,None,None,None,None,None,None,None,None,590.104574


Applying filters:

In [40]:
def filter_list(filter1, filter2):
    l = list()
    l.append(filter1)
    l.append(filter2)
    return l

In [41]:
filterList = filter_list('SATMT25','SATVR25')
filterList

['SATMT25', 'SATVR25']

In [42]:
def filter_values(filter1,filter1input, filter2, filter2input):
    filters_dict = {}
    filters_dict[filter1] = filter1input
    filters_dict[filter2] = filter2input
    return filters_dict

In [52]:
filterValues = filter_values('SATMT25', '586', 'SATVR25', '789')
filterValues

{'SATMT25': '586', 'SATVR25': '789'}

In [71]:
#stuck on getting argument df to work in format function
def apply_filters(df, featuresDict, filterList, filterValues):
    filterquery = """
    SELECT
    {1},
    {2},
    {3}
    FROM {0}
    WHERE {4} <= {5}
    AND {6} <= {7}
    """.format(df,featuresDict['feature1'],featuresDict['feature2'],featuresDict['feature3'], filterList[0], filterValues[filterList[0]],filterList[1],  filterValues[filterList[1]])
    return sqldf(filterquery,locals())

In [72]:
appliedfilters = apply_filters(df=test,featuresDict = featuresDict,filterList=filterList,filterValues=filterValues)
appliedfilters

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U1'), dtype('float64')) -> None

In [73]:

    filterquery = """
    SELECT
    UNITID
    FROM {0}
    """.format("'" + test
    sqldf(filterquery,locals())

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U1'), dtype('float64')) -> None

In [64]:
test

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,INSTURL,NPCURL,HCM2,...,OMENRYP_PELL_FT_POOLED_SUPP,OMENRAP_PELL_FT_POOLED_SUPP,OMAWDP8_PELL_FT_POOLED_SUPP,OMENRUP_PELL_FT_POOLED_SUPP,OMENRYP_PELL_PT_POOLED_SUPP,OMENRAP_PELL_PT_POOLED_SUPP,OMAWDP8_PELL_PT_POOLED_SUPP,OMENRUP_PELL_PT_POOLED_SUPP,GT_THRESHOLD_P6_SUPP,dist_miles
0,100654,00100200,001002,Alabama A & M University,Normal,AL,35762,www.aamu.edu/,www.aamu.edu/admissions-aid/tuition-fees/net-p...,0,...,0.0116,0.3329,0.2958,0.3597,0.0056,0.3889,0.1833,0.4222,0.4606,147.593425
1,100663,00105200,001052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,https://www.uab.edu/,https://tcc.ruffalonl.com/University of Alabam...,0,...,0.0149,0.2773,0.5243,0.1835,0.0211,0.277,0.4354,0.2665,0.659,156.540810
2,100690,02503400,025034,Amridge University,Montgomery,AL,36117-3553,www.amridgeuniversity.edu/,www2.amridgeuniversity.edu:9091/,0,...,0,0.2798,0.2523,0.4679,0,0.186,0.4186,0.3953,0.5132,164.191256
3,100706,00105500,001055,University of Alabama in Huntsville,Huntsville,AL,35899,www.uah.edu/,finaid.uah.edu/,0,...,0.0098,0.3154,0.4961,0.1787,0.0105,0.3632,0.3474,0.2789,0.717,150.129854
4,100724,00100500,001005,Alabama State University,Montgomery,AL,36104-0271,www.alasu.edu/,www.alasu.edu/cost-aid/tuition-costs/net-price...,0,...,0.0097,0.3904,0.3061,0.2938,0.0778,0.4444,0.1111,0.3667,0.382,169.201972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6676,49576719,00332917,003329,Pennsylvania State University-Penn State Wilke...,Lehman,PA,18627-0217,wilkesbarre.psu.edu/,None,0,...,None,None,None,None,None,None,None,None,None,670.943595
6677,49576720,00332918,003329,Pennsylvania State University-Penn State York,York,PA,17403-3326,york.psu.edu/,None,0,...,None,None,None,None,None,None,None,None,None,577.729643
6678,49576721,00332919,003329,Pennsylvania State University-Penn State Great...,Malvern,PA,19355-1488,greatvalley.psu.edu/,None,0,...,None,None,None,None,None,None,None,None,None,630.970517
6679,49576722,00332920,003329,Pennsylvania State University-Penn State Harri...,Middletown,PA,17057-4846,harrisburg.psu.edu/,None,0,...,None,None,None,None,None,None,None,None,None,590.104574


Normalizing Features

creating features dict

In [45]:
def features_dict(feature1, feature2, feature3):
    featuresDict = {}
    featuresDict['feature1']= feature1
    featuresDict['feature2']= feature2
    featuresDict['feature3']= feature3
    return featuresDict

In [46]:
featuresDict = features_dict('OMENRUP_PELL_NFT_POOLED_SUPP', 'OMENRYP_PELL_FT_POOLED_SUPP', 'OMENRAP_PELL_FT_POOLED_SUPP')

Applying Ranking Algo

grabbing feature columns

In [ ]:
featuresQuery = """
SELECT
UNITID, {}, {}, {}
FROM inst
""".format(featuresDict['x1'],featuresDict['x2'],featuresDict['x3'])

In [ ]:
featuresdf2 = pd.read_sql(featuresQuery,conn)

In [ ]:
test = pd.merge(featuresDF, dist_temp_table, on='UNITID')
test

Normalizing each feature

part 1: find each feature's length

In [ ]:
maxQuery = """
SELECT max({0}) x1FeatureLength, max({1}) x2FeatureLength, max({2}) x3FeatureLength
FROM inst
where {0} not like 'PrivacySuppressed' and {1} not like 'PrivacySuppressed' and {2} not like 'PrivacySuppressed'
""".format(featuresDict['x1'],featuresDict['x2'],featuresDict['x3'])

In [ ]:
featureLengthDF = sqldf(maxQuery,locals())

In [ ]:
featureLengths = featureLengthDF.loc[0]

In [ ]:
x1FeatureLength = float(test['x1FeatureLength'])
x2FeatureLength = float(test['x2FeatureLength'])
x3FeatureLength = float(test['x3FeatureLength'])

part 2: applying length to normalize each feature

In [ ]:
normalizedQuery = """
SELECT
{0}/{3} feature1normalized,
{1}/{4} feature2normalized,
{2}/{5} feature3normalized
FROM inst
""".format(featuresDict['x1'],featuresDict['x2'],featuresDict['x3'], x1FeatureLength, x2FeatureLength, x3FeatureLength)

In [ ]:
normalizedDF = sqldf(normalizedQuery,locals())
normalizedDF

multiplying preferences to feature

In [ ]:
prefVector = np.array([0.3,0.2,0.5])

In [ ]:
normalizedMatrix = normalizedDF.values

In [ ]:
normalizedMatrix

In [ ]:
ratingScore = np.dot(normalizedMatrix, prefVector)

In [ ]:
baseDF['Rating'] = ratingScore

In [ ]:
testQuery = """
SELECT count(*)
FROM baseDF
where rating is not NULL
"""

In [ ]:
output = sqldf(testQuery,locals())
output

apply filters